# Praktikum 2

Authors: Ahmed Khalil and Fiona Lublow

## Research question:

**How well can we approximate Images with voronoi diagrams evolved with a genetic algorithm?**

## Theory
Somewhat similiar example project: https://github.com/damdoy/genetic_image_approximation

Good info about genetic algorithm for Images: https://medium.com/@sebastian.charmot/genetic-algorithm-for-image-recreation-4ca546454aaa

Genetic algorithm tutorial: https://www.datacamp.com/tutorial/genetic-algorithm-python

Voronoi Algorithms:
- https://stackoverflow.com/questions/64101510/converting-voronoi-diagram-cell-areas-to-lists-of-pixel-coordinates
- https://svn.osgeo.org/qgis/trunk/qgis/python/plugins/fTools/tools/voronoi.py
- jump flooding algorithm: https://www.comp.nus.edu.sg/~tants/jfa/i3d06.pdf

- fast but not 100% precise voronoi algorithm: https://en.wikipedia.org/wiki/Jump_flooding_algorithm
    - GPU-Accelerated implementation (Maciej A. Czyzewski & Kamil Piechowiak): https://github.com/maciejczyzewski/fast_gpu_voronoi
    - Documentation: https://pypi.org/project/fast-gpu-voronoi/

Adjacent paper about polygonal approximation: https://doi.org/10.1016/S0167-8655(98)00082-8

Color comparison with delta E CIE1976: http://zschuessler.github.io/DeltaE/learn/

## Description of experiment

### Individuums in our population:
We need to represent pictures as Individuums with genes that we can mutate and crossover

Every Individuum should be one whole picture, maybe there is a way to represent areas of images as well, but we did not explore this further here.

### Voronoi Diagrams
A [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) is a partition of a plane into regions, which are defined by points and their corresponding color on the plane.
The whole area of the plane will be colored, with each pixel gaining the color of its closest point, so each region in the plane will be defined by the location and the color of a point.

Using Voronoi diagrams, we can make arbitrarily complex pictures with points for polygons.

We can represent an image as a set of points and their colors, each of these points is one gene in our Individuum, each gene has a 2d coordinate, and a 3-Tuple for RGB-Color.

or instead of points: with number of points, and distributions (x and y distinct distribution of points, distribution of colours) <- this seems way more random not sure if thats good

calculating the fitness of an individuum might be difficult, since points need to be converted with a voronoi algorithm to pixel images and then compared

-> we have an optimization opportunity (parallelization / use GPU for this)


### Genetic Algorithm aspects (Selection, Crossover and Mutation)

- Selection: we can use standard methods here (Tournament, Roulette-wheel selection, etc.)

- Crossover is easy, just randomly or (50:50) take points from either image

- Possible mutations : probabilities for different mutations -> hyper parameters

    - changing coordinates of points by some amount
    - changing color by some amount: one or multiple of RGB up/down
    - adding points (we might need upper bound, or put amount as a factor in fitness function, else more points might always be better)
    - removing point (naive approach: just delete it. Maybe better: take two neighbours and fuse them into one (coordinates and colours as average of fusion inputs)
We have to take care when we adjust values while mutating, need to check for overflow outside of picture and outside of 8bit range for colors

Different solutions possible:
- set to minimum/maximum
- set a new random value
- overflow to the other side of the range (x = -1 goes to x = width etc)


In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import colour # this has to be installed from conda-forge, the package is called colour-science!
import random
import numpy.random as npr
import concurrent.futures
import time

In [ ]:
# Define image size
IMAGE_SIZE = (64, 64)

In [ ]:
# Voronoi Algorithm
# this code slightly modified from https://gist.github.com/pv/8036995
def voronoi_to_image(point_color_list: np.ndarray) -> np.ndarray:
    """
    Generates a Voronoi diagram rendered as an image of a specified size.

    Parameters:
    ----------
    point_color_list : list of tuples
        A list where each entry is a tuple of (x, y, color), with color being an RGB tuple.

    Returns:
    -------
    img : np.ndarray
        A NumPy array of shape (height, width, 3) representing the RGB image.
    """

    def voronoi_finite_polygons_2d(vor, radius=None):
        """
        Helper function to reconstruct infinite Voronoi regions into finite regions.
        """
        if vor.points.shape[1] != 2:
            raise ValueError("Requires 2D input")

        new_regions = []
        new_vertices = vor.vertices.tolist()

        center = vor.points.mean(axis=0)
        if radius is None:
            radius = vor.points.ptp().max() * 2

        # Construct a map containing all ridges for a given point
        all_ridges = {}
        for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
            all_ridges.setdefault(p1, []).append((p2, v1, v2))
            all_ridges.setdefault(p2, []).append((p1, v1, v2))

        # Reconstruct infinite regions
        for p1, region in enumerate(vor.point_region):
            vertices = vor.regions[region]

            if all(v >= 0 for v in vertices):
                # finite region
                new_regions.append(vertices)
                continue

            # reconstruct a non-finite region
            ridges = all_ridges[p1]
            new_region = [v for v in vertices if v >= 0]

            for p2, v1, v2 in ridges:
                if v2 < 0:
                    v1, v2 = v2, v1
                if v1 >= 0:
                    # finite ridge: already in the region
                    continue

                # Compute the missing endpoint of an infinite ridge
                t = vor.points[p2] - vor.points[p1]  # tangent
                t /= np.linalg.norm(t)
                n = np.array([-t[1], t[0]])  # normal

                midpoint = vor.points[[p1, p2]].mean(axis=0)
                direction = np.sign(np.dot(midpoint - center, n)) * n
                far_point = vor.vertices[v2] + direction * radius

                new_region.append(len(new_vertices))
                new_vertices.append(far_point.tolist())

            # sort region counterclockwise
            vs = np.asarray([new_vertices[v] for v in new_region])
            c = vs.mean(axis=0)
            angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
            new_region = np.array(new_region)[np.argsort(angles)]

            # finish
            new_regions.append(new_region.tolist())

        return new_regions, np.asarray(new_vertices)

    # Extract points and colors
    points = np.array([p[:2] for p in point_color_list])  # Extract (x, y) coordinates
    colors = np.array([p[2] for p in point_color_list])  # Extract colors (RGB tuples)

    # Compute Voronoi diagram
    #vor = Voronoi(points)

    # Map points to pixel grid
    pixel_coords = (points * [IMAGE_SIZE[1] - 1, IMAGE_SIZE[0] - 1]).astype(int)

    # Create blank image
    img = np.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype=np.float32)

    # Render Voronoi diagram pixel by pixel
    for y in range(IMAGE_SIZE[0]):
        for x in range(IMAGE_SIZE[1]):
            # Find the closest Voronoi point to the pixel
            distances = np.linalg.norm(pixel_coords - [x, y], axis=1)
            closest_point_idx = np.argmin(distances)
            img[y, x] = colors[closest_point_idx]

    # Convert to valid image format (0-255)
    img = np.asarray(img * 255, dtype=np.uint8)
    return img

In [ ]:
# Our implementation of the Jump Flooding Algorithm
# sadly it's slower than the other algorithm

def jfa_voronoi(points, image_size, max_steps=None):
    height, width = image_size
    # Speichert die Koordinaten der nächstgelegenen Punkte für jedes Pixel im Bild
    # Wird mit -1 initialisiert, um zu zeigen, dass Pixel noch keinem Punkt zugeordnet wurde
    # 2 Dimensionen, in diesem Fall x und y Koordinaten eines Punktes
    
    grid = np.full((height, width, 2), -1, dtype=int)
    distance = np.full((height, width), np.inf)
    
    # Skalierung der Punkte auf das Bild (normierte Punkte in Pixel umwandeln)
    scaled_points = (points * np.array([width, height])).astype(int)
    
    for idx, (x, y) in enumerate(scaled_points):
        grid[y, x] = [x, y]  # Koordinaten des Punktes zuweisen
        distance[y, x] = 0   # Entfernung auf 0 setzen
    
    max_dim = max(width, height)
    max_jump = int(2 ** np.ceil(np.log2(max_dim)))
    
    if max_steps is None:
        max_steps = int(np.log2(max_jump)) + 1
    
    jump = max_jump
    while jump >= 1:
        for y in range(height):
            for x in range(width):
                if grid[y, x][0] == -1:
                    continue
                
                # Iteriere über benachbarte Pixel (dx, dy)
                for dy in [-jump, 0, jump]:
                    for dx in [-jump, 0, jump]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < width and 0 <= ny < height:
                            source = grid[y, x]
                            dist = np.sqrt((nx - source[0]) ** 2 + (ny - source[1])**2)
                            if dist < distance[ny, nx]:
                                grid[ny, nx] = source
                                distance[ny, nx] = dist
        
        jump //= 2  # Sprungweite halbieren
    
    return grid

# NEU 
def voronoi_to_image_jfa(point_color_list, image_size=(100, 100)):
    # Ensure that point_color_list is a list of tuples (x, y, color)
    points = np.array([p[:2] for p in point_color_list])  # Extract the (x, y) coordinates
    colors = np.array([p[2] for p in point_color_list])  # Extract the RGB color values

    # Berechne Voronoi mit Jump Flooding Algorithmus
    grid = jfa_voronoi(points, image_size)
    height, width = image_size
    img = np.zeros((height, width, 3), dtype=np.float32)
    
    # Finde für jedes Pixel den nächsten Punkt und weise ihm die Farbe zu
    for y in range(height):
        for x in range(width):
            closest_point_idx = np.argmin(np.linalg.norm(points - grid[y, x] / np.array(image_size), axis=1))
            img[y, x] = colors[closest_point_idx]
    
    img = np.asarray(img * 255, dtype=np.uint8)
    return img


# Test with random points and colors
test_points = [
    (0.2, 0.3, (1.0, 0.0, 0.0)),  # Red
    (0.8, 0.2, (0.0, 1.0, 0.0)),  # Green
    (0.4, 0.9, (0.0, 0.0, 1.0)),  # Blue
    (0.7, 0.6, (1.0, 1.0, 0.0)),  # Yellow
]

# Generate Voronoi image
image_size = (200, 200)
voronoi_img = voronoi_to_image_jfa(test_points, image_size)

# Show image
plt.imshow(voronoi_img)
plt.title("Voronoi Diagram via JFA")
plt.axis('off')
plt.show()

In [ ]:
# fitness function
# delta_E is a smart way to compare color differences in images, which is closer to human perception of differences than basic mean squared error or similar

def delta_e(image1, image2):
    return np.mean(colour.difference.delta_e.delta_E_CIE1976(image1, image2)) * -1

# helper function for standardizing fitness for roulette-wheel based selection
def normalize_fitness(fitnesses: np.array) -> list[float]:
    fitnesses_array = np.array(fitnesses, dtype=float)
    min_fitness = np.min(fitnesses_array)
    max_fitness = np.max(fitnesses_array)

    if max_fitness - min_fitness > 0:
        normalized_fitnesses = (fitnesses_array - min_fitness) / (max_fitness - min_fitness)
    else:
    # If all fitness values are the same, set normalized values to 0.5 or any constant
        normalized_fitnesses = np.full_like(fitnesses_array, 0.5)

    return list(normalized_fitnesses)

In [ ]:
# Functions for elements of the genetic algorithm
# general structure is inspired by this: https://www.datacamp.com/tutorial/genetic-algorithm-python

# random initialization of one individuum
def initialize_with_points(num_points):
    points = []
    for i in range(num_points):
        x, y, r, g, b = np.random.rand(5)
        point = (x, y, tuple([r, g, b]))
        points.append(point)
    return points

# initializing a population
def create_initial_population(size: int, num_points: int) -> list[np.array]:
    population = []
    for i in range(size):
        individual = initialize_with_points(num_points)
        population.append(individual)
    return population

# SELECTION
def tournament_selection(population: list[np.array], fitnesses: list[float], tournament_size: int =10) -> list[np.array]:
    selected = []
    for i in range(len(population)):
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament = [(population[i], fitnesses[i]) for i in tournament_indices]

        # Find the individual with the best fitness in the tournament
        winner = max(tournament, key=lambda x: x[1])  # Higher fitness is better
        selected.append(winner[0])
    return selected

# this requires a normalized fitnesses list
def roulette_selection(population: list[np.array], fitnesses: list[float]) -> list[np.array]:
    fitnesses = normalize_fitness(fitnesses)
    selected = []
    max = sum(fitnesses)
    selection_probs = [fitness/max for fitness in fitnesses]
    for i in range(len(population)):
        selected.append(population[npr.choice(len(population), p=selection_probs)])
    return selected

# this is computationally the worst (because sorting) but using it for testing purposes,
# fitness calculation takes many times as long anyway
def sorted_selection(population: list[np.array], fitnesses: list[float]) -> list[np.array]:
    paired = list(zip(population, fitnesses))
    # Sort the paired list by fitness in descending order
    sorted_paired = sorted(paired, key=lambda x: x[1], reverse=True)

    # Select the top individuals based on num_to_select
    selected = [individual for individual, _ in sorted_paired]
    selected = selected + selected # duplicate the selected half
    random.shuffle(selected)
    return selected

# CROSSOVER
def crossover_2_to_2(parent1: np.array, parent2: np.array) -> (np.array, np.array):
    # shuffle the parents to eliminate always having genes in same position
    np.random.shuffle(parent1)
    np.random.shuffle(parent2)
    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1) - 1)
    # Create children by swapping slices at the crossover point
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# MUTATION
# Lots of values to tweak here, rate is how often mutation occurs, range is how far it can mutate (1 = basically just assign a new value)
# there might be a problem here, where if a value <lower_bound of > upper_bound is chosen, it just sets it to bound, leads groups at edges
# lower and upper bound necessary like this, cause values exactly at the border make that voronoi function freak out

def add_random_point(individual: np.array, lower_bound=0.001, upper_bound=0.999) -> np.array:
    new_point = (
        random.uniform(lower_bound, upper_bound),
        random.uniform(lower_bound, upper_bound),
        (
            random.uniform(0, 1),
            random.uniform(0, 1),
            random.uniform(0, 1),
        ),
    )
    return individual + [new_point]


def remove_random_point(individual: np.array) -> np.array:
    # Individuums with one point are not viable and cause problems in the parallelized fitness function
    if len(individual) > 2:
        remove_idx = random.randint(0, len(individual) - 1)
        return individual[:remove_idx] + individual[remove_idx+1:]
    return individual

# adjust values instead of settings new
def mutation_values_adjusted(individual: np.array, mutation_rate: float, lower_bound: float =0.001, upper_bound:float =0.999,
                             color_mutation_range: float =1, coordinate_mutation_range: float =0.3) -> np.array:
    mutated_individual = []

    for point in individual:
        x, y, (r, g, b) = point

        # Mutate (x, y)
        if random.random() < mutation_rate:
            x_mutation = random.uniform(-coordinate_mutation_range, coordinate_mutation_range)
            x += x_mutation
            x = max(min(x, upper_bound), lower_bound)

        if random.random() < mutation_rate:
            y_mutation = random.uniform(-coordinate_mutation_range, coordinate_mutation_range)
            y += y_mutation
            y = max(min(y, upper_bound), lower_bound)

        # Mutate (r, g, b)
        if random.random() < mutation_rate:
            color = random.sample([r, g, b], 1) # this only chooses ONE color if it mutates, might be better to mutate all ?
            color_mutation = random.uniform(-color_mutation_range, color_mutation_range)
            if color == r:
                r += color_mutation
                r = max(min(r, upper_bound), lower_bound)
            elif color == g:
                g += color_mutation
                g = max(min(g, upper_bound), lower_bound)
            elif color == b:
                b += color_mutation
                b = max(min(b, upper_bound), lower_bound)

        mutated_individual.append((x, y, (r, g, b)))

    return mutated_individual


def mutation_with_add_remove(individual: np.array, mutation_rate: float) -> np.array:
    individual = mutation_values_adjusted(individual, mutation_rate)
    
    if random.random() < mutation_rate:
        individual = add_random_point(individual)
        
    if random.random() < mutation_rate and len(individual) > 1:
        individual = remove_random_point(individual)
    
    return individual

# if mutation occurs, set the values anew instead of adjusting
def mutation_values_set_anew(individual: np.array, mutation_rate: float) -> np.array:
    mutated_individual = []
    for point in individual:
        x, y, (r, g, b) = point

        # point
        if random.random() < mutation_rate:
            x, y = np.random.rand(2)

        # color
        if random.random() < mutation_rate:
            r, g, b = np.random.rand(3)

        mutated_individual.append((x, y, (r, g, b)))
    mutated_individual = mutation_with_add_remove(mutated_individual, mutation_rate * 1.5)

    return mutated_individual

In [ ]:
# Driver of the genetic algorithm

# selecting and adjusting the target image
target_image = Image.open("./data/matterhorn2_small.png")
target_image = target_image.convert("RGB")
target_image = target_image.resize(IMAGE_SIZE)
target_image_array = np.asarray(target_image, dtype="uint8")

# the functions to use are set here
fitness_fn = delta_e
mutation_fn = mutation_values_set_anew

def genetic_algorithm(mutation_rate_start: float, population: int, num_points: int, generations: int):
    target_fitness = fitness_fn(target_image_array, target_image_array)

    print(f"Starting Evolution with: {population} population, {num_points} points per individuum, {generations} generations")
    mutation_rate = mutation_rate_start / num_points

    population = create_initial_population(population, num_points)

    best_image = None
    average_calc_time = 0

    for generation in range(generations):
        start = time.perf_counter()
        # FITNESS
        fitnesses = parallel_calculate_fitness(population, target_image_array)

        fitnesses = fitnesses + [target_fitness] # include target_fitness to normalize including that
        normalized = normalize_fitness(fitnesses) # normalize for handling in selection_fn

        fitnesses = normalized[:-1]  # drop target_fitness via slice

        # Get the best fitness and individual
        best_fitness = max(fitnesses)
        best_individual = population[fitnesses.index(best_fitness)]

        # we could add a break condition with no fitness change for x generations
        # or increase mutation rate when no change for a while

        best_image = voronoi_to_image(best_individual)

        # show the progress every 10 epochs
        visualize(10, best_fitness, best_image, generation, average_calc_time)

        # SELECTION
        population = roulette_selection(population, fitnesses)

        # CROSSOVER and MUTATION
        next_population = parallel_crossover_and_mutate(population, mutation_rate)

        # Preserve the best individual
        next_population[0] = best_individual
        population = next_population
        end = time.perf_counter()
        average_calc_time = (average_calc_time + end - start) / 2
    return best_image


# parallelization helper
def calculate_fitness(individual: np.array, target_image_array: np.ndarray) -> float:
    try:
        return fitness_fn(target_image_array, voronoi_to_image(individual))
    except Exception as e:
        print(f"Error in calculate_fitness: {e}")
        return float('inf')  # Fehlerhafte Individuen ausschließen

def parallel_calculate_fitness(population: list[np.array], target_image_array: np.ndarray) -> list[float]:
    with concurrent.futures.ProcessPoolExecutor() as executor:
        fitnesses = list(executor.map(calculate_fitness, population, [target_image_array] * len(population)))
    return fitnesses


# parallelization helper
def crossover_and_mutate(individual1: np.array, individual2: np.array, mutation_rate: float) -> (np.array, np.array):
    child1, child2 = crossover_2_to_2(individual1, individual2)
    child1 = mutation_fn(child1, mutation_rate)
    child2 = mutation_fn(child2, mutation_rate)
    return child1, child2

def parallel_crossover_and_mutate(population: list[np.array], mutation_rate: float) -> list[np.array]:
    # Parallelizing crossover and mutation for next population
    next_population = []
    # Pairwise crossover and mutation, slice on even/odd idx
    p1s = population[::2]
    p2s = population[1::2]

    for ind in population:
        if(len(ind) <= 1):
            print(ind)

    mutation_rates = np.empty(len(population)) # array for ProcessPoolThread (why necessary?)
    mutation_rates.fill(mutation_rate)

    with concurrent.futures.ProcessPoolExecutor() as executor:
        children = list(executor.map(crossover_and_mutate, p1s, p2s, mutation_rates))

    # Flatten the list of tuples (child1, child2)
    for child1, child2 in children:
        next_population.append(child1)
        next_population.append(child2)
    return next_population


def visualize(increment, best_fitness, best_image, generation, average_calc_time):
    if generation % increment == 0:
        fig, axs = plt.subplots(1, 2, sharey=True, sharex=True)  # Create side-by-side plot
        axs[0].imshow(target_image)
        axs[0].set_title(f"Target Image\nGeneration {generation}\navg Calc Time/gen {average_calc_time:.4f}")
        if best_image is not None:
            axs[1].imshow(best_image)
            axs[1].set_title(f"Current Best Image\nFitness: {best_fitness:.4f}")
        else:
            axs[1].imshow(target_image)
        for ax in axs:
            ax.axis("off")
        fig.show()
        plt.pause(0.01)

In [ ]:
# these settings take about 0.5s/generation on Fionas PC, and only a little longer on haw jupyterhub
solution = genetic_algorithm(0.1, 100, 6, 500)
# show the final result
plt.imshow(solution)
plt.show()

## Result

We can approximate simple images very well, but detailed images with a lot of little elements are only roughly approximated, more than about 10 regions in our voronoi diagram lead to bad results, at least with < 250 generations

## Reflection

Creating the image from the points and calculating its fitness was our bottleneck, with each generation taking too much time we were only able to run our algorithm for a couple hundred generations.

Comparing two images is more complex than just taking the difference in RGB values, we saw a big improvement using the delta_E visual perception comparison

We tried to parallelize more aspects of our functions, especially the voronoi calculation with jump flooding was explored and attempted to implement on the GPU via Numba, this did not work out in thef time-scope of this project, but might have led to a big improvement of runtime.